### connecting mysql server

In [90]:
import mysql.connector
from faker import Faker
import random

In [92]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name):
    """
    Creates a connection to the MySQL database.

    Args:
        host_name: The hostname of the MySQL server.
        user_name: The username for the MySQL database.
        user_password: The password for the MySQL user.
        db_name: The name of the database to connect to.

    Returns:
        A MySQL connection object if successful, None otherwise.
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

# Replace with your actual database credentials
host = "localhost"  # Corrected: Hostname is usually "localhost"
user = "cdac" 
password = "password" 
database = "dbms" 

# Create a connection
db = create_connection(host, user, password, database)

# Close the connection if it was established
if db:
    db.close()
    print("MySQL connection closed.")

Connection to MySQL DB successful
MySQL connection closed.


In [94]:
# Initialize Faker and database connection
fake = Faker()
db = mysql.connector.connect(
    host="localhost",
    user="cdac",
    password="password",
    database="dbms"
)
cursor = db.cursor()




In [95]:
# Step 1: Create Tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT PRIMARY KEY AUTO_INCREMENT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    sign_up_date DATE,
    region VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Products (
    product_id INT PRIMARY KEY AUTO_INCREMENT,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10, 2)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT PRIMARY KEY AUTO_INCREMENT,
    customer_id INT,
    order_date DATE,
    order_status VARCHAR(20),
    total_amount DECIMAL(10, 2),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS OrderDetails (
    order_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);
""")



In [98]:
# Step 2: Populate Customers Table
regions = ['North', 'South', 'East', 'West']
for _ in range(1000):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = f"{first_name.lower()}.{last_name.lower()}@{fake.domain_name()}"
    sign_up_date = fake.date_between(start_date='-5y', end_date='today')
    region = random.choice(regions)

    cursor.execute("""
    INSERT INTO Customers (first_name, last_name, email, sign_up_date, region)
    VALUES (%s, %s, %s, %s, %s)
    """, (first_name, last_name, email, sign_up_date, region))

db.commit()



In [99]:
# Step 3: Populate Products Table
categories = ['Electronics', 'Clothing', 'Home Goods', 'Books', 'Toys']
for _ in range(100):
    product_name = f"{fake.word().capitalize()} {fake.word().capitalize()}"
    category = random.choice(categories)
    price = round(random.uniform(5.00, 1000.00), 2)

    cursor.execute("""
    INSERT INTO Products (product_name, category, price)
    VALUES (%s, %s, %s)
    """, (product_name, category, price))

db.commit()



In [100]:
from decimal import Decimal
import random
from faker import Faker

# ... (your existing code for creating database connection) ...

fake = Faker()

for _ in range(2500):
    customer_id = random.randint(1, 1000)
    order_date = fake.date_between(start_date='-1y', end_date='today')
    order_status = random.choice(['Completed', 'Pending', 'Canceled'])
    total_amount = Decimal('0.00')

    cursor.execute("""
        INSERT INTO Orders (customer_id, order_date, order_status, total_amount)
        VALUES (%s, %s, %s, %s)
    """, (customer_id, order_date, order_status, total_amount))

    order_id = cursor.lastrowid

    num_products = random.randint(1, 5)
    for _ in range(num_products):
        product_id = random.randint(1, 100)
        quantity = random.randint(1, 10)

        cursor.execute("SELECT price FROM Products WHERE product_id = %s", (product_id,))
        price = cursor.fetchone()[0]  # Keep price as Decimal
        total_amount += price * quantity

        # Check for existing entry before inserting
        cursor.execute("""
            SELECT * FROM OrderDetails
            WHERE order_id = %s AND product_id = %s
        """, (order_id, product_id))

        if not cursor.fetchone():  # No existing entry found
            cursor.execute("""
                INSERT INTO OrderDetails (order_id, product_id, quantity)
                VALUES (%s, %s, %s)
            """, (order_id, product_id, quantity))
        else:
            print(f"Duplicate entry found for order_id: {order_id} and product_id: {product_id}")

    cursor.execute("""
        UPDATE Orders
        SET total_amount = %s
        WHERE order_id = %s
    """, (total_amount, order_id))

db.commit()  # Corrected indentation
print("Data inserted successfully.")



Duplicate entry found for order_id: 7532 and product_id: 26
Duplicate entry found for order_id: 7535 and product_id: 21
Duplicate entry found for order_id: 7552 and product_id: 74
Duplicate entry found for order_id: 7555 and product_id: 70
Duplicate entry found for order_id: 7581 and product_id: 24
Duplicate entry found for order_id: 7582 and product_id: 51
Duplicate entry found for order_id: 7591 and product_id: 77
Duplicate entry found for order_id: 7609 and product_id: 91
Duplicate entry found for order_id: 7705 and product_id: 76
Duplicate entry found for order_id: 7731 and product_id: 25
Duplicate entry found for order_id: 7763 and product_id: 18
Duplicate entry found for order_id: 7794 and product_id: 49
Duplicate entry found for order_id: 7819 and product_id: 7
Duplicate entry found for order_id: 7918 and product_id: 73
Duplicate entry found for order_id: 7975 and product_id: 41
Duplicate entry found for order_id: 7976 and product_id: 67
Duplicate entry found for order_id: 8007 

In [101]:
# Check inserted data
print("\nChecking Inserted Data:")

# Check Orders table
cursor.execute("SELECT * FROM Orders LIMIT 10") 
print("\nOrders Table:")
for row in cursor:
    print(row)

# Check OrderDetails table
cursor.execute("SELECT * FROM OrderDetails LIMIT 10")
print("\nOrderDetails Table:")
for row in cursor:
    print(row)

cursor.close()
db.close()

print("\nDatabase populated successfully!")
print("Database connection closed.")


Checking Inserted Data:

Orders Table:
(1, 766, datetime.date(2024, 1, 21), 'Canceled', Decimal('0.00'))
(2, 841, datetime.date(2024, 11, 30), 'Canceled', Decimal('986.65'))
(3, 29, datetime.date(2024, 7, 29), 'Canceled', Decimal('939.09'))
(4, 445, datetime.date(2024, 1, 4), 'Pending', Decimal('6665.60'))
(5, 186, datetime.date(2024, 3, 14), 'Pending', Decimal('635.28'))
(6, 76, datetime.date(2024, 2, 18), 'Completed', Decimal('1369.00'))
(7, 60, datetime.date(2024, 12, 6), 'Completed', Decimal('1369.90'))
(8, 103, datetime.date(2024, 8, 20), 'Pending', Decimal('8043.57'))
(9, 920, datetime.date(2024, 3, 24), 'Completed', Decimal('3161.04'))
(10, 52, datetime.date(2024, 9, 9), 'Canceled', Decimal('5390.91'))

OrderDetails Table:
(2, 2, 1)
(2, 33, 4)
(2, 75, 3)
(2, 82, 2)
(2, 84, 9)
(3, 21, 7)
(3, 83, 1)
(4, 53, 9)
(4, 87, 8)
(4, 90, 7)

Database populated successfully!
Database connection closed.
